In [ ]:
%matplotlib notebook
import numpy as np
from scipy.integrate import odeint, quad
from scipy.optimize import brentq
import matplotlib.pyplot as plt
from matplotlib import animation, rc
import seaborn as sbs
rc('font', **{'family': 'serif', 'serif': ['Computer Modern'], 'size': 20})
rc('text', usetex=True)
rc('animation', html='html5')

In [ ]:
# The potential and its first derivative, as callables.
V = lambda x: 0.5 * x**2 * (0.5 * x**2 - 1)
dVdx = lambda x: x**3 - x

In [ ]:
# The potential energy function on a grid of x-points.
xgrid = np.linspace(-1.5, 1.5, 100)
Vgrid = V(xgrid)

In [ ]:
plt.plot(xgrid, Vgrid)
plt.xlabel('$x$')
plt.ylabel('$V(x)$')

In [ ]:
def deriv(X, t, gamma, delta, omega):
    """Return the derivatives dx/dt and d2x/dt2."""
    
    x, xdot = X
    xdotdot = -dVdx(x) -delta * xdot + gamma * np.cos(omega*t)
    return xdot, xdotdot

def solve_duffing(tmax, dt_per_period, t_trans, x0, v0, gamma, delta, omega):
    """Solve the Duffing equation for parameters gamma, delta, omega.
    
    Find the numerical solution to the Duffing equation using a suitable
    time grid: tmax is the maximum time (s) to integrate to; t_trans is
    the initial time period of transient behaviour until the solution
    settles down (if it does) to some kind of periodic motion (these data
    points are dropped) and dt_per_period is the number of time samples
    (of duration dt) to include per period of the driving motion (frequency
    omega).
    
    Returns the time grid, t (after t_trans), position, x, and velocity,
    xdot, dt, and step, the number of array points per period of the driving
    motion.
    
    """
    # Time point spacings and the time grid

    period = 2*np.pi/omega
    dt = 2*np.pi/omega / dt_per_period
    step = int(period / dt)
    t = np.arange(0, tmax, dt)
    # Initial conditions: x, xdot
    X0 = [x0, v0]
    X = odeint(deriv, X0, t, args=(gamma, delta, omega))
    idx = int(t_trans / dt)
    return t[idx:], X[idx:], dt, step

In [ ]:
# Set up the motion for a oscillator with initial position
# x0 and initially at rest.
x0, v0 = 0, 0
tmax, t_trans = 18000, 300
omega = 1
gamma, delta = 0.15, 0.25
dt_per_period = 100

In [ ]:
# Solve the equation of motion.
t, X, dt, pstep = solve_duffing(tmax, dt_per_period, t_trans, x0, v0, gamma, delta, omega)
x, xdot = X.T
print(len(x))

In [ ]:
# The animation
fig, ax = plt.subplots(nrows=2,ncols=2)
ax1 = ax[0,0]
ax1.plot(xgrid, Vgrid)
ln1, = ax1.plot([], [], 'mo')
ax1.set_xlabel(r'$x / \mathrm{m}$')
ax1.set_ylabel(r'$V(x) / \mathrm{J}$')

# Position as a function of time
ax2 = ax[1,0]
ax2.set_xlabel(r'$t / \mathrm{s}$')
ax2.set_ylabel(r'$x / \mathrm{m}$')
ln2, = ax2.plot(t[:100], x[:100])
ax2.set_ylim(np.min(x), np.max(x))

# Phase space plot
ax3 = ax[1,1]
ax3.set_xlabel(r'$x / \mathrm{m}$')
ax3.set_ylabel(r'$\dot{x} / \mathrm{m\,s^{-1}}$')
ln3, = ax3.plot([], [])
ax3.set_xlim(np.min(x), np.max(x))
ax3.set_ylim(np.min(xdot), np.max(xdot))

# Poincaré section plot
ax4 = ax[0,1]
ax4.set_xlabel(r'$x / \mathrm{m}$')
ax4.set_ylabel(r'$\dot{x} / \mathrm{m\,s^{-1}}$')
ax4.scatter(x[::pstep], xdot[::pstep], s=2, lw=0)
scat1 = ax4.scatter([x0], [v0], lw=0, c='m')

plt.tight_layout()

def animate(i):
    """Update the image for iteration i of the Matplotlib animation."""
    
    ln1.set_data(x[i], V(x[i]))
    ln2.set_data(t[:i+1], x[:i+1])
    ax2.set_xlim(t_trans, t[i])
    ln3.set_data(x[:i+1], xdot[:i+1])
    if not i % pstep:
        scat1.set_offsets(X[i])
    return

anim = animation.FuncAnimation(fig, animate, frames=len(x), interval=1)

plt.show()

Writer = animation.writers['ffmpeg']
writer = Writer(fps=15, metadata=dict(artist='Me'), bitrate=180)

anim.save('duffing.mp4', writer=writer)

In [ ]:
def H(x, p):
    return p**2 / 2 + V(x)


X, P = np.meshgrid(x, p)
Z = H(X, P)

plt.figure()
plt.xlabel('$x$', fontdict=font)
plt.ylabel('$p$', fontdict=font)
plt.tick_params(labelsize=lblsize)
plt.contour(X, P, Z, 30, colors='black')
d = np.ma.array(Z, mask=Z==0)
plt.contour(X, P, d, 0, colors='red')
plt.show()

In [ ]:
plt.close()
x0, v0 = 1e-7, 0

# Solve the equation of motion.
t, X1, dt, pstep = solve_duffing(tmax, dt_per_period, t_trans, x0, v0, gamma, delta, omega)
xtv, xdottv = X1.T


# The animation
fig, ax = plt.subplots(nrows=2,ncols=2)
ax1 = ax[0,0]
ax1.plot(xgrid, Vgrid)
ln1, = ax1.plot([], [], 'mo')
ax1.set_xlabel(r'$x / \mathrm{m}$')
ax1.set_ylabel(r'$V(x) / \mathrm{J}$')

# Position as a function of time
ax2 = ax[1,0]
ax2.set_xlabel(r'$t / \mathrm{s}$')
ax2.set_ylabel(r'$x / \mathrm{m}$')
ln2, = ax2.plot(t[:100], xtv[:100])
ax2.set_ylim(np.min(xtv), np.max(xtv))

# Phase space plot
ax3 = ax[1,1]
ax3.set_xlabel(r'$x / \mathrm{m}$')
ax3.set_ylabel(r'$\dot{x} / \mathrm{m\,s^{-1}}$')
ln3, = ax3.plot([], [])
ax3.set_xlim(np.min(xtv), np.max(xtv))
ax3.set_ylim(np.min(xdottv), np.max(xdottv))

# Poincaré section plot
ax4 = ax[0,1]
ax4.set_xlabel(r'$x / \mathrm{m}$')
ax4.set_ylabel(r'$\dot{x} / \mathrm{m\,s^{-1}}$')
ax4.scatter(xtv[::pstep], xdottv[::pstep], s=2, lw=0)
scat1 = ax4.scatter([x0], [v0], lw=0, c='m')

plt.tight_layout()

anim1 = animation.FuncAnimation(fig, animate, frames=len(xtv), interval=1)

plt.show()

In [ ]:
dx = [xt - xtvt for xt, xtvt in zip(x, xtv)]
dp = [xdott - xdottvt for xdott, xdottvt in zip(xdot, xdottv)]

Dist = [np.sqrt(dx[i]**2 + dp[i]**2) for i in range(len(dx))]

LogDist = np.log(Dist)

plt.figure()
plt.xlabel('$t$')
plt.ylabel('$dist$')
plt.plot(Dist)
plt.show()

plt.figure()
plt.xlabel('$t$')
plt.ylabel('$\log(dist)$')
plt.plot(LogDist)
plt.axhline(y=0, color='k', linewidth=0.5)
plt.show()

print(LogDist[-1]/len(t))

In [ ]:
from scipy import signal
import numpy as np
import matplotlib.pyplot as plt

fs = 10e3
N = 1e5
amp = 2*np.sqrt(2)
time = t / fs
x = np.asarray(x)


# np.fft.fft
time_step = 1 / 10
freqs = np.fft.fftfreq(x.size, time_step)
idx = np.argsort(freqs)
ps = np.abs(np.fft.fft(x))**2
plt.figure()
plt.plot(freqs[idx], ps[idx])
plt.title('Power spectrum (np.fft.fft)')

# signal.welch
f, Pxx_spec = signal.welch(x, fs, 'flattop', 1024, scaling='spectrum')
plt.figure()
plt.semilogy(f, np.sqrt(Pxx_spec))
plt.xlabel('frequency [Hz]')
plt.ylabel('Linear spectrum [V RMS]')
plt.title('Power spectrum (scipy.signal.welch)')
plt.show()

In [ ]:
from __future__ import division

data = np.asarray(x)
ps = np.abs(np.fft.fft(data))**2

time_step = 1 / 10
freqs = np.fft.fftfreq(data.size, time_step)
idx = np.argsort(freqs)

plt.figure()
plt.plot(freqs[idx], ps[idx])
plt.show()